In [18]:
from datasets import load_from_disk
import numpy as np
import soundfile as sf
import os, sys

from sklearn.model_selection import train_test_split

SEED=42
SAMPLE_RATE=16000
LABELS = ['other', 'drone']

ds = load_from_disk("../data/datasets/dataset1")

In [19]:
print(ds)
print(f"Features: {ds.features}")
print(f"Number of samples: {len(ds)}")

Dataset({
    features: ['audio', 'label'],
    num_rows: 191770
})
Features: {'audio': Audio(sampling_rate=None, decode=True, stream_index=None), 'label': ClassLabel(names=['other', 'drone'])}
Number of samples: 191770


In [23]:
df = ds.to_pandas()
# Split into train (70%), temporary(30%)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=SEED, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED, stratify=temp_df['label'])

In [24]:
print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))
print("\nClass distribution in train:")
print(train_df['label'].value_counts())
print("\nClass distribution in validation:")
print(val_df['label'].value_counts())
print("\nClass distribution in test:")
print(test_df['label'].value_counts())

Train size: 134239
Validation size: 28765
Test size: 28766

Class distribution in train:
label
1    118662
0     15577
Name: count, dtype: int64

Class distribution in validation:
label
1    25427
0     3338
Name: count, dtype: int64

Class distribution in test:
label
1    25428
0     3338
Name: count, dtype: int64


In [37]:
import soundfile as sf

# Load a sample audio file
sample = ds[60000]
sampling_rate = sample['audio']['sampling_rate']
audio_array = sample['audio']['array']

print("Sampling rate:", sampling_rate)
print("Audio shape:", audio_array.shape)
print("Duration (seconds):", len(audio_array) / sampling_rate)
print("Label: ", sample['label'])

from IPython.display import Audio as DisplayAudio
display(DisplayAudio(audio_array, rate=sampling_rate))

Sampling rate: 16000
Audio shape: (8000,)
Duration (seconds): 0.5
Label:  1
